In [1]:
import boto3
import json
import time

In [2]:
lex = boto3.client('lex-models', region_name='us-east-1')
lambd = boto3.client('lambda', region_name='us-east-1')
iam = boto3.client('iam', region_name='us-east-1')


In [3]:
import sagemaker
from sagemaker import get_execution_role

aws_role = get_execution_role()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [4]:
aws_role

'arn:aws:iam::287758680514:role/service-role/AmazonSageMaker-ExecutionRole-20231027T004238'

In [5]:
def wait(secs):
    time.sleep(secs)

In [7]:
role_policy_document = {
        u'Version': u'2012-10-17',
        u'Statement': [
            {u'Effect': u'Allow', u'Principal': {u'Service': [u'lambda.amazonaws.com']},
             u'Action': [u'sts:AssumeRole']},
        ]
    }

In [8]:
role_policy_document

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': ['lambda.amazonaws.com']},
   'Action': ['sts:AssumeRole']}]}

In [9]:
role = iam.create_role(RoleName='songChatbot-role', AssumeRolePolicyDocument=json.dumps(role_policy_document))['Role']
print ('Created custom role shakirachatbot-role')
wait(3)

ClientError: An error occurred (AccessDenied) when calling the CreateRole operation: User: arn:aws:sts::287758680514:assumed-role/AmazonSageMaker-ExecutionRole-20231027T004238/SageMaker is not authorized to perform: iam:CreateRole on resource: arn:aws:iam::287758680514:role/shakirachatbot-role because no identity-based policy allows the iam:CreateRole action

In [4]:
inline_policy = {
    u'Version': u'2012-10-17',
    u'Statement': [
        {
            u'Effect': u'Allow',
            u'Action': [
                u'polly:SynthesizeSpeech',
                u's3:ListBucket',
                u's3:PutObject',
                u's3:PutObjectAcl',
                u'logs:CreateLogGroup',
                u'logs:CreateLogStream',
                u'logs:PutLogEvents'
            ],
            u'Resource': u'*'
        }
    ]
}


In [5]:
inline_policy

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Action': ['polly:SynthesizeSpeech',
    's3:ListBucket',
    's3:PutObject',
    's3:PutObjectAcl',
    'logs:CreateLogGroup',
    'logs:CreateLogStream',
    'logs:PutLogEvents'],
   'Resource': '*'}]}

In [13]:
iam.put_role_policy(RoleName='AmazonSageMaker-ExecutionRole-20231027T004238', PolicyName='BotAccess', PolicyDocument=json.dumps(inline_policy))
print ('Created inline policy for role shakirachatbot-role')


Created inline policy for role shakirachatbot-role


In [6]:
lex = boto3.client('lex-models', region_name='us-east-1')
lambd = boto3.client('lambda', region_name='us-east-1')
iam = boto3.client('iam', region_name='us-east-1')

In [11]:
from zipfile import ZipFile

with ZipFile('lambda-package.zip', 'w') as myzip:
    myzip.write('chatbot/__init__.py')
    myzip.write('chatbot/botcontrol.py')

In [6]:
# abc = open('lambda-package.zip').read().decode('utf-8')
import zipfile

file_content = ""
# Open the ZIP file
with zipfile.ZipFile('lambda-package.zip', 'r') as zip_ref:
    # List all file names in the zip
    for file_name in zip_ref.namelist():
        # print(f'Reading file: {file_name}')
        
        # Read each file
        with zip_ref.open(file_name) as file:
            # If it's a text file, you need to know the correct encoding
            # If you're not sure, you might have to try different encodings
            file_content += file.read().decode('utf-8')  # or 'cp1252', 'iso-8859-1', etc.
            # print(file_content)

In [10]:
with open('./lambda-package.zip', 'rb') as f:
    zip_file_content = f.read()

In [12]:
# zip_file_content

In [18]:
import os
FUNCTION_NAME = 'Song-Botcontrol'
ACCOUNT_ID=287758680514
ROLE_NAME = 'AmazonSageMaker-ExecutionRole-20231027T004238'
botcontrol_function = {
    u'Code': {u'ZipFile': zip_file_content},
    u'Description': u'Function controling the Songs Chatbot, powered up by Lex',
    u'FunctionName': u'{}'.format(FUNCTION_NAME),
    u'Handler': u'chatbot/botcontrol.lambda_handler',
    u'MemorySize': 128,
    u'Role': 'arn:aws:iam::287758680514:role/basic-lambda-execute',
    u'Runtime': u'python3.10',
    u'Timeout': 10,
    u'TracingConfig': {u'Mode': u'PassThrough'},
    u'Publish': True,
}

In [16]:
lambda_role =  u'arn:aws:iam::{}:role/{}'.format(ACCOUNT_ID,ROLE_NAME)

In [17]:
lambda_role

'arn:aws:iam::287758680514:role/AmazonSageMaker-ExecutionRole-20231027T004238'

In [19]:
lambd.create_function(**botcontrol_function)
print ('Created lambda function {}'.format(FUNCTION_NAME))
wait(3)

Created lambda function Song-Botcontrol
